In [ ]:
from zk_adventures_types import F, Polynomial

In [ ]:
n = 5

MultilinearPolynomialXY = PolynomialRing(F, 2 * n, [f"X{i}" for i in range(n)] + [f"Y{i}" for i in range(n)])

MultilinearPolynomial = PolynomialRing(F, n, [f"X{i}" for i in range(n)])

X_powers = [MultilinearPolynomial(f"X{i}") for i in range(n)]

In [ ]:
def compute_L_H():
    L_H_factors = [MultilinearPolynomialXY(f"1 + X{i} * Y{i}") for i in range(n)]

    result = pow(2, -n, F.order())
    for factor in L_H_factors:
        result *= factor
    return result

In [ ]:
values = [F.random_element() for _ in range(8)]

In [ ]:
def index_to_hypercube(i, k):
    return [1 if (i >> (k - l - 1)) & 1 else -1 for l in range(k)]

index_to_hypercube(6, 3)

In [ ]:
def interpolate_values(values):
    L_H = compute_L_H()

    result = F(0)
    for i, f_i in enumerate(values):
        result += f_i * L_H(*X_powers, *index_to_hypercube(i, n))
    return result

In [ ]:
poly = interpolate_values(values)

assert poly(*index_to_hypercube(7, n)) == values[7]

In [ ]:
def sum_polynomial_over_hypercube(polynomial):
    result = F(0)
    for i in range(2**n):
        result += polynomial(*index_to_hypercube(i, n))
    return result

In [ ]:
s = sum_polynomial_over_hypercube(poly)

In [ ]:
def compute_Si(polynomial, rs):
    i = len(rs) + 1
    X = Polynomial.monomial(1)
    result = F(0)
    for k in range(2**(n - i)):
        # variables = (r_1, ..., r_{i-1}, X, x_{i+1}, ..., x_n)
        variables = rs + [X] + index_to_hypercube(k, n - i)
        result += polynomial(*variables)
    return result

In [ ]:
random_values = []

In [ ]:
s1 = compute_Si(poly, random_values)
s == s1(1) + s1(-1)

In [ ]:
r1 = F.random_element()
random_values.append(r1)
s2 = compute_Si(poly, random_values)
s1(r1) == s2(1) + s2(-1)

In [ ]:
r2 = F.random_element()
random_values.append(r2)
s3 = compute_Si(poly, random_values)
s2(r2) == s3(1) + s3(-1)

In [ ]:
r3 = F.random_element()
random_values.append(r3)
s4 = compute_Si(poly, random_values)
s3(r3) == s4(1) + s4(-1)

In [ ]:
r4 = F.random_element()
random_values.append(r4)
s5 = compute_Si(poly, random_values)
s4(r4) == s5(1) + s5(-1)

In [ ]:
r5 = F.random_element()
random_values.append(r5)
s5(r5) == poly(*random_values)

In [ ]:
from tqdm.auto import tqdm

# INTERACTIVE SUMCHECK

random_values = []

# prover
s_i = compute_Si(poly, random_values)

# verifier
assert s == s_i(1) + s_i(-1)

for i in tqdm(range(1, n)):
    # verifier
    r_i = F.random_element()
    
    # prover
    random_values.append(r_i)
    s_i_plus_1 = compute_Si(poly, random_values)
    
    # verifier
    assert s_i(r_i) == s_i_plus_1(1) + s_i_plus_1(-1)
    s_i = s_i_plus_1

# verifier
r_last = F.random_element()
random_values.append(r_last)
s_i(r_last) == poly(*random_values)